In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [0]:
def extraeDataSenamhi(code_estation, year_month, type_est, estado):
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/_dato_esta_tipo02.php"
    new_url = "{}?estaciones={}&CBOFiltro={}&t_e={}&estado={}".format(url,
                                                                     code_estation,
                                                                     year_month,
                                                                     type_est,
                                                                     estado)
    df_tmp = pd.read_html(new_url, header=0)
    df = df_tmp[0]
    if type(df.iloc[0,1]) == str:
        df = df.drop(df.index[0])
    return df

In [0]:
def main(code_estation, year_month, type_est, estado):
    ldates = [str(y) + str(x).zfill(2) for x in range(1,13) for y in range(year_month[0], year_month[1])]
    ldates.sort()
    data = []
    for l in ldates:
        try:
            df = extraeDataSenamhi(code_estation, l, type_est, estado)
            if type(data) == list:
                data = pd.DataFrame({}, columns = list(df.columns))
            else:
                data = data.append(df)
            print(l)
        except:
            pass
    return data

In [4]:
l = main("212600", [2010, 2016], "H", "AUTOMATICA")

201401
201402
201403
201404
201504
201505
201506
201507
201508
201509
201510
201511
201512


In [0]:
# d["cod"], [1980, 2020], d["ico"], d["estado"]

In [0]:
import json
with open('estaciones.json', 'r', encoding='cp1252') as f:
    data = json.load(f)

In [20]:
data["estaciones"]

[{'cate': 'CO',
  'cod': '100090',
  'cod_old': '000396',
  'estado': 'DIFERIDO',
  'ico': 'M',
  'lat': -7.22499,
  'lon': -79.15323,
  'nom': 'MONTE GRANDE'},
 {'cate': 'PLU',
  'cod': '106067',
  'cod_old': '153109',
  'estado': 'DIFERIDO',
  'ico': 'M',
  'lat': -6.27366,
  'lon': -79.03696,
  'nom': 'QUEROCOTILLO'},
 {'cate': 'HLG',
  'cod': '221110',
  'cod_old': '220307',
  'estado': 'DIFERIDO',
  'ico': 'H',
  'lat': -6.67411,
  'lon': -78.52437,
  'nom': 'PUENTE  MAYGASBAMBA'},
 {'cate': 'PLU',
  'cod': '100113',
  'cod_old': '153108',
  'estado': 'DIFERIDO',
  'ico': 'M',
  'lat': -6.68778,
  'lon': -78.45694,
  'nom': 'QUEBRADA SHUGAR'},
 {'cate': 'PLU',
  'cod': '106077',
  'cod_old': '153208',
  'estado': 'DIFERIDO',
  'ico': 'M',
  'lat': -6.67091,
  'lon': -78.73717,
  'nom': 'CHUGUR'},
 {'cate': 'CO',
  'cod': '105062',
  'cod_old': '000349',
  'estado': 'DIFERIDO',
  'ico': 'M',
  'lat': -5.67175,
  'lon': -78.89767,
  'nom': 'LA CASCARILLA'},
 {'cate': 'CO',
  'cod': 